# Prediction Tendency Task - Translation into Python

### Author: Merle Schuckart
### Contact: merle.schuckart@uni-luebeck.de
### Version: 9th of May, 2023
-----

#### What is this notebook about?
I would like to include Juliane Schubert's prediction tendency task in my EEG study EXNAT-2, and she was so kind to give me access to her experiment repo on Gitlab.

In her task, she "presented tone sequences of varying entropy level, to independently quantify auditory prediction tendency (as the tendency to anticipate low-level acoustic features) for each individual." (Schubert et al., 2023 --> https://doi.org/10.1093/cercor/bhac528)

Now the "problem" is that the original prediction tendency task is completely built in Matlab, whereas I wanted to have 1 single experiment file completely written in Python and not different sub-experiments in different programming languages.

So I have to translate more or less everything into Python. I use this notebook to collect and test all the snippets before I add them to the experiment.

Structure of Julianes Repo:

withinblock_markov
  - exp
      - init
          - config.ptb.m
          - create_markov_transition_matrix.m
          - init.ptb.m
          - prepare_cfg.m
          - prepare_stims.m
      - parts
          - helper
              - get_trigger_sequence.m
              - prepare_markov_sound_chain.m
              - submit_markov_sound_chain.m
          - block.m
          - prepare_subject.m
          - resting.m
          - volume_bayes.m
  - functions
      - create_markov_matrix_from_entropy.m
      - create_mid_markov.m
      - create_ordered_markov.m
      - create_random_markov.m
      - ft_cfg2keyval.m
      - ft_getopt.m
  - o_ptb_helper
      - volume_bayes.m
      - volume_staircase.m
      - private
          - volume_bayes_present.m
  - external
      --> a lot of external package files?
  - calc_nstims.m
  - run.m
  - sync_behav.sh

In [40]:
# import packages
import numpy as np
from scipy.optimize import fmin_slsqp # for minimizing constrained nonlinear multivariable functions - I think that's the Python alternative for Matlab's fmincon function
import matplotlib.pyplot as plt # for testing

!pip install hmmlearn
from hmmlearn import hmm # for hidden markov models

from scipy.optimize import fmin_constrained

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ImportError: ignored

In [38]:
''' Helper functions '''
# Some Matlab functions don't exist in Python (at least not 1:1),
# so I wrote custom ones.



# Function: Find minimum of constrained nonlinear multivariable function
#def





# Entropy Error Function:
def entropy_error(x, entropy):
  print("running function entropy_error (from custom helper functions)")

  p = np.reshape(x, (n_features, n_features))  # Reshape the input vector to matrix form
  return np.sum(p * np.log(p)) - entropy  # Compute the entropy error using the reshaped matrix form of the input vector




In [ ]:
''' Python Code for "exp/init/config.ptb.m" '''

# Those are just settings for the experiment, I guess I don't need them.

In [16]:
''' Python Code for "exp/init/init.ptb.m" '''
# I guess this is just to set up triggers and sound devices?


' Python Code for "exp/init/init.ptb.m" '

In [37]:
# DONE
''' Python Code for "exp/init/prepare_cfg.m" '''


''' What does this function do? '''
# --> creates dict called cfg and fills it with the
#     specified values. Matlab's struct = Python's dict

def prepare_cfg():
    print("running function prepare_cfg")

    # This function prepares paths and configures
    # stimulus properties and trigger values

    cfg = {}
    cfg['data_path'] = 'data'

    cfg['markov'] = {}
    cfg['markov']['n_features'] = 4
    cfg['markov']['min_prob'] = 0.05
    cfg['markov']['iti'] = 1/3
    cfg['exp'] = {}
    cfg['exp']['trials_per_block'] = 1500

    cfg['add_db'] = 40

    base_freq = 440
    fourth_ratio = 4/3

    freqs = []
    for idx_freq in range(1, 5):
        freqs.append(base_freq * (fourth_ratio**(idx_freq-1)))

    cfg['sounds'] = {}
    cfg['sounds']['freqs'] = freqs
    cfg['sounds']['stim_length'] = 0.1
    cfg['sounds']['fade'] = 5e-3

    cfg['triggers'] = {}
    cfg['triggers']['stims'] = [16, 32, 64, 128]

    cfg['triggers']['condition'] = {}
    cfg['triggers']['condition']['ordered'] = 1
    cfg['triggers']['condition']['random'] = 2

    # note that triggers will be added for every stimulus to provide info about
    # current entropy level! (e.g. F440 during random = 17)
    return cfg


# test function:
print(prepare_cfg())
# This is how you extract certain values (e.g. number of features):
print(prepare_cfg()['markov']['n_features'])

running function prepare_cfg
{'data_path': 'data', 'markov': {'n_features': 4, 'min_prob': 0.05, 'iti': 0.3333333333333333}, 'exp': {'trials_per_block': 1500}, 'add_db': 40, 'sounds': {'freqs': [440.0, 586.6666666666666, 782.2222222222222, 1042.9629629629628], 'stim_length': 0.1, 'fade': 0.005}, 'triggers': {'stims': [16, 32, 64, 128], 'condition': {'ordered': 1, 'random': 2}}}
running function prepare_cfg
4


In [64]:
# DONE?

# --> relies on create_ordered_markov(), create_random_markov()
# and create_mid_markov() from "functions/" file

''' Python Code for "exp/init/create_markov_transition_matrix.m" '''

def create_markov_transition_matrix(cfg):
    print("running function create_markov_transition_matrix")

    # Loads all markov trans_mats
    n_features = cfg['markov']['n_features']
    trans_mats = {}

    trans_mats['ordered'] = create_ordered_markov(n_features)
    trans_mats['random'] = create_random_markov(n_features)
    trans_mats['mid'] = create_mid_markov(n_features)

    return trans_mats

print(create_markov_transition_matrix(prepare_cfg()))

running function prepare_cfg
running function create_markov_transition_matrix
[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
{'ordered': array([[0.25, 0.75, 0.  , 0.  ],
       [0.  , 0.25, 0.75, 0.  ],
       [0.  , 0.  , 0.25, 0.75],
       [0.  , 0.  , 0.  , 0.25]]), 'random': array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), 'mid': array([[0.375, 0.375, 0.   , 0.25 ],
       [0.25 , 0.375, 0.375, 0.   ],
       [0.   , 0.25 , 0.375, 0.375],
       [0.375, 0.   , 0.25 , 0.375]])}


In [69]:
# DONE
# --> relies on create_markov_transition_matrix() from exp/init/...

''' Python Code for "exp/init/prepare_stims.m" '''


''' What does this function do? '''

# 1. Creates audio stimuli using the cfg configuration object and
#    returns a dict of audio stimuli and their trigger values.

#    The audio stimuli are created by looping over the freqs in cfg["sounds"]["freqs"],
#    building sine waves for the current freq and duration,
#    and applying a cosine ramp to each stimulus.
#    The trigger names are from cfg["triggers"]["stims"].


# 2. Generates Markov transition matrix (using the create_markov_transition_matrix
#    function from the exp.init module) & add it to the
#    output dict under the name "markov".

#     Function Argument:
#             - cfg object (output from the custom prepare_cfg() function)

#     Function Output:
#             - dict called "stims" containing audio stimuli and
#               their trigger values + Markov transition matrix

def prepare_stims(cfg):

    print("running function prepare_stims")

    stims = {"audio": [], "trigger": [], "markov": {}}

    # loop frequencies in cfg["sounds"]["freqs"]
    for i in range(len(cfg["sounds"]["freqs"])):

        # get current frequency:
        cur_freq = cfg["sounds"]["freqs"][i]
        print("cur_freq:", cur_freq)
        # build audio stimulus by creating a sine wave with the current frequency and duration
        # careful, o_ptb is from a MATLAB package: https://o-ptb.readthedocs.io/en/latest/
        #audio_stim = o_ptb.stimuli.auditory.Sine(cur_freq, cfg["sounds"]["stim_length"])

        # get duration of current stimulus from dict
        cur_duration = cfg["sounds"]["stim_length"]
        print("cur_duration:", cur_duration)
        # build a time array: you need the current duration and the right sampling frequency for your device
        # 1 divided by the sampling rate = duration of a single sample in sec
        audio_sample_freq = 44100 # 44100 Hz --> audio sampling rate at the lab (according to Frauke)
        sound_sample_len = 1/audio_sample_freq
        t = np.arange(0, cur_duration, sound_sample_len)

        # generate sine wave:
        sine_wave = np.sin(2*np.pi*cur_freq*t)

        # plot the sine wave
        #plt.plot(t, sine_wave)
        #plt.xlabel('Time (s)')
        #plt.ylabel('Amplitude')
        #plt.show()

        # Apply cosine ramp to "smoothen" the edges of the sound a bit (I'm not an audio expert as you can tell)
        # We basically gradually turn up the sound, play it for a while,
        # and then decrease the volume again so it doesn't make annoying clicky noises when it's played.

        # get fade duration from config
        cur_fade_duration = cfg["sounds"]["fade"]
        print("fade:", cur_fade_duration)

        # apply cosine ramp:
        # check how many samples we have to use for the fade in/out:
        fade_samples = int(cur_fade_duration * audio_sample_freq)

        # if there are enough, but not too many fade samples,
        # apply cosine ramp to signal
        if fade_samples > 0 and fade_samples < len(sine_wave):
            ramp = np.cos(np.linspace(0, np.pi / 2, fade_samples))
            sine_wave[:fade_samples] *= ramp[::-1]
            sine_wave[-fade_samples:] *= ramp

        # plot the modified sine wave again
        #plt.plot(t, sine_wave)
        #plt.xlabel('Time (s)')
        #plt.ylabel('Amplitude')
        #plt.show()

        # append audio stimulus and corresponding trigger to "stims" dict
        stims["audio"].append(sine_wave)
        stims["trigger"].append(cfg["triggers"]["stims"][i])

    # also add markov transition matrix to the output dict
    # under the name "trans_mats"
    #print(create_markov_transition_matrix(cfg))
    stims["markov"]["trans_mats"] = create_markov_transition_matrix(cfg)

    # return dict with stims and markov transition matrix:
    return stims


# Test it:
print("stims: ", prepare_stims(prepare_cfg()))

running function prepare_cfg
running function prepare_stims
cur_freq: 440.0
cur_duration: 0.1
fade: 0.005
cur_freq: 586.6666666666666
cur_duration: 0.1
fade: 0.005
cur_freq: 782.2222222222222
cur_duration: 0.1
fade: 0.005
cur_freq: 1042.9629629629628
cur_duration: 0.1
fade: 0.005
running function create_markov_transition_matrix
[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
stims:  {'audio': [array([ 0.00000000e+00,  4.49346638e-04,  1.79380974e-03, ...,
       -2.68190203e-03, -8.96927941e-04, -3.83610348e-18]), array([ 0.00000000e+00,  5.98823598e-04,  2.38687039e-03, ...,
       -1.02546001e-02, -5.52826173e-03, -5.02875253e-17]), array([0.00000000e+00, 7.97708208e-04, 3.17095573e-03, ...,
       1.25271019e-02, 6.61349319e-03, 5.87454165e-17]), array([0.00000000e+00, 1.06189786e-03, 4.20067389e-03, ...,
       1.41729241e-02, 7.17238186e-03, 6.06135064e-17])], 'trigger': [16, 32, 64, 128], 'markov': {'trans_mats': {'ordered': array([[0.

In [34]:
# DONE?
# --> relies on input sequence, condition, trigger


''' Python Code for "exp/parts/helper/get_trigger_sequence.m" '''

def get_trigger_sequence(sequence, condition, trigger):

    print("running function get_trigger_sequence")

    ''' What does this function do?'''
    # - transforms item values to trigger values using the trigger
    #   info from cfg.triggers
    # - Condition triggers will be added to stimulus
    #   triggers to provide info about the current entropy level.


    trigger_sequence = np.zeros_like(sequence)

    for i in range(len(sequence)):
        cur_stim = sequence[i]
        cur_trigger = trigger["stims"][cur_stim] + trigger["condition"][condition]
        trigger_sequence[i] = cur_trigger

    print("generated a sequence of triggers:", trigger_sequence)
    return trigger_sequence


In [ ]:
# DONE?
# --> relies on cfg, stims, condition, n_stims
''' Python Code for "exp/parts/helper/prepare_markov_sound_chain.m" '''


''' What does this function do? '''
# The markov chain (with length n_stims) is generated from transition
# probabilities (hidden to the observer, as we have defined them) and
# emission probabilities (= probabilities of the observations themselves)

# EXAMPLE: suppose we have only 2 states = (tones)
# p_trans = [0, 1;... # from state 1 it goes always to state 2
#           1, 0]     # (and vice versa)

# p_em = [0.9, 0.1;... # BUT if state 1 is chosen there is still a small prob that state 2 will be emitted
#        0.1, 0.9]     # (and vice versa)

# markov_chain = hmmgenerate(10,p_trans,p_em) # this is matlab code btw
# the generated sequence could look like this:
# 2     1     2     1     1     1     2     1     2     1

# in our case we always want to emit the given state with 100% probability.


def prepare_markov_sound_chain(cfg, stims, condition, n_stims):
    print("running function prepare_markov_sound_chain")

    # Generate the Markov chain sequence based on
    # transition and emission probabilities

    # Get the transition matrix based on the given condition
    transition_matrix = stims["markov"]["trans_mats"][condition]

    # Define the emission probability matrix as an
    # identity matrix since we want to emit
    # the given state with 100% probability
    emission_matrix = np.eye(cfg["markov"]["n_features"])

    # get settings from cfg
    n_features = cfg["markov"]["n_features"]

    # Generate the Markov chain sequence using the transition and emission matrices
    model = hmm.MultinomialHMM(n_components = n_features,
                               verbose = False,
                               params = "ste")
    model.startprob_ = np.ones(n_features) / n_features  # Set equal initial probabilities
    model.transmat_ = transition_matrix
    model.emissionprob_ = emission_matrix
    sequence = model.sample(n_samples = n_stims, random_state = None)[0].tolist()

# Test it!
cfg = prepare_cfg()
stims = prepare_stims(cfg)
condition = "ordered"
n_stims = 10

sequence = prepare_markov_sound_chain(cfg, stims, condition, n_stims)
print(sequence)



In [ ]:
''' Python Code for "exp/parts/helper/submit_markov_sound_chain.m" '''




In [ ]:
''' Python Code for "exp/parts/block.m" '''




In [ ]:
''' Python Code for "exp/parts/prepare_subject.m" '''




In [ ]:
''' Python Code for "exp/parts/resting.m" '''




In [ ]:
''' Python Code for "exp/parts/volume_bayes.m" '''




In [62]:
''' Python Code for "functions/create_markov_matrix_from_entropy.m" '''

''' What does this function do? '''
# --> create a markov transition matrix, i.e. a matrix of
#     n x n values (so basically a square matrix)
#     that reflect a certain target entropy distribution.
#     Each value is somewhere between 0 and 1, with the values in each row adding up to 1.

#     At the beginning of the function, a random Markov matrix is created.
#     After this, a minimisation algorithm (fmincon from the MATLAB optimization toolbox)
#     is used to modify the values in the matrix so the target entropy distribution is reached.

#     Function Arguments:
#            n_features = number of states or features to be modelled
#            entropy = target entropy distribution
#            zero_elements = number of zero elements (default: 0)
#            low_bound = lowest possible value for the matrix values
#            optim_options = optimization options

#     Function Output:
#            A Markov transition matrix that should match the target entropy distribution.




def create_markov_matrix_from_entropy(n_features, entropy, zero_elements = 0, low_bound = 0, optim_options = None):

    print("running function create_markov_matrix_from_entropy")

    # Use default optimizer options if they are not set in the function call:
    if optim_options == None:
      # set options for optimizer
      optim_options = {'maxiter': 30000} # max. number of function evaluations allowed is 30000

    # Create a square matrix of shape n x n with
    # random values between 0 and 1
    startmat = np.random.rand(n_features, n_features)





    # Set the lower and upper bounds of the optimization variables
    lower_bound = np.zeros((n_features, n_features)) + low_bound
    np.fill_diagonal(lower_bound, 1 / n_features)  # Set the diagonal elements to 1/n_features
    upper_bound = np.ones((n_features, n_features))
    np.fill_diagonal(upper_bound, 1 / n_features)  # Set the diagonal elements to 1/n_features



    # Set some elements of the upper and lower
    # bounds to zero to force some transitions to be zero

    # loop number of zero elements:
    for i in range(zero_elements):
        # loop columns by looping number of features
        for cur_col in range(n_features):
            # get row index:
            idx = cur_col + i
            # if index is >= the number of columns...
            if idx >= n_features: # careful, this is > in the original function as Matlab starts counting from 1, not from 0 as in Python
                # set new row index by subtracting total number of columns from current index (this should set idx as 0 I think)
                idx = idx - n_features
            # now set upper and lower bound at row idx and current column index as 0
            upper_bound[idx, cur_col] = 0
            lower_bound[idx, cur_col] = 0


    # Define the nonlinear constraints for the optimization problem
    nonlcon = lambda x: (sum(x) - 1) + (np.sum(x, axis=1) - 1), []

    # Use fmincon to minimize the entropy error function subject to the constraints
    trans_mat = fmincon(lambda x: entropy_error(x, entropy), startmat.ravel(), [], [], [], [], lower_bound.ravel(), upper_bound.ravel(), nonlcon, options=optim_options)

    # Reshape the output vector to matrix form
    trans_mat = np.reshape(trans_mat, (n_features, n_features))

    return trans_mat



In [61]:
# DONE
''' Python Code for "functions/create_mid_markov.m" '''

def create_mid_markov(n_features):

    print("running function create_mid_markov")

    first_row = np.zeros(n_features)
    first_row[0] = 1/n_features  # 0.25 repetition

    n_remaining = n_features - 2
    p_remaining = (1 - first_row[0]) / n_remaining  # equal probability of 0.375 for next or jump
    first_row[1:-1] = p_remaining

    trans_mat = np.zeros((n_features, n_features))

    for idx_row in range(n_features):
        trans_mat[idx_row, :] = np.roll(first_row, idx_row-1)

    return trans_mat

# Test it:
print(create_mid_markov(n_features = 4))


[[0.375 0.375 0.    0.25 ]
 [0.25  0.375 0.375 0.   ]
 [0.    0.25  0.375 0.375]
 [0.375 0.    0.25  0.375]]


In [60]:
# DONE
''' Python Code for "functions/create_ordered_markov.m" '''

def create_ordered_markov(n_features):

    print("running function create_ordered_markov")

    # matrix with 0.25 along the diagonal axis
    trans_mat = np.eye(n_features) * 1/n_features
    print(trans_mat)

    # add 0.75 probability for the next tone
    # (basically the values in the matrix after the 0.25 values)
    # loop rows:
    for cur_row in range(n_features-1):
        idx = cur_row + 1

        if idx > n_features:
            idx = idx - n_features

        trans_mat[cur_row, idx] = 1 - 1/n_features

    return trans_mat

# Test it:
print(create_ordered_markov(n_features = 4))


[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
[[0.25 0.75 0.   0.  ]
 [0.   0.25 0.75 0.  ]
 [0.   0.   0.25 0.75]
 [0.   0.   0.   0.25]]


In [59]:
# DONE
''' Python Code for "functions/create_random_markov.m" '''

def create_random_markov(n_features):

    print("running function create_random_markov")

    trans_mat = np.ones((n_features, n_features)) * 1/n_features

    return trans_mat

# Test it:
print(create_random_markov(n_features = 4))


[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]


In [58]:
''' Python Code for "functions/ft_cfg2keyval.m" '''


''' What does this function do?'''
# The function ft_cfg2keyval takes a structure cfg
# and converts it into a cell-array called "optarg"
# containing key-value pairs. If the input cfg is empty, an empty
# list is returned.
# I think Juliane got this from the FIELDTRIP toolbox.
# I don't think
def ft_cfg2keyval(cfg):

    print("running function ft_cfg2keyval")

    # Convert a structure to a cell-array with key-value pairs
    if cfg:
        keys = list(cfg.keys())
        values = list(cfg.values())
        optarg = [keys, values]
        optarg = [item for sublist in optarg for item in sublist]
    else:
        optarg = []
    return optarg

# Test it_
print(prepare_cfg()) # before tranformation
print("--------")
print(ft_cfg2keyval(prepare_cfg())) # after tranformation

running function prepare_cfg
{'data_path': 'data', 'markov': {'n_features': 4, 'min_prob': 0.05, 'iti': 0.3333333333333333}, 'exp': {'trials_per_block': 1500}, 'add_db': 40, 'sounds': {'freqs': [440.0, 586.6666666666666, 782.2222222222222, 1042.9629629629628], 'stim_length': 0.1, 'fade': 0.005}, 'triggers': {'stims': [16, 32, 64, 128], 'condition': {'ordered': 1, 'random': 2}}}
--------
running function prepare_cfg
['data_path', 'markov', 'exp', 'add_db', 'sounds', 'triggers', 'data', {'n_features': 4, 'min_prob': 0.05, 'iti': 0.3333333333333333}, {'trials_per_block': 1500}, 40, {'freqs': [440.0, 586.6666666666666, 782.2222222222222, 1042.9629629629628], 'stim_length': 0.1, 'fade': 0.005}, {'stims': [16, 32, 64, 128], 'condition': {'ordered': 1, 'random': 2}}]


In [56]:
# DONE I guess?! I don't know why I need this.
''' Python Code for "functions/ft_getopt.m" '''


''' What does this function do?'''
# retrieves the value of a specified option key
# from a configuration structure opt or a cell-array.
# If the key is present in the structure or cell-array,
# the corresponding value will be returned.
# If the key is not present, the function will return
# the default value.
# The "emptymeaningful" parameter specifies whether
# an empty value should be considered meaningful or not.
# If it is True, an empty value will be returned as is.
# If it is False, the default value will be
# returned instead.


def ft_getopt(opt, key, default = None, emptymeaningful = False):
    # Get the value of a specified option from a configuration structure or cell-array

    if default is None:
        default = []

    if opt is None:
        return default

    if isinstance(opt, dict):
        # Get the key-value from the structure
        if key not in opt:
            val = default
        else:
            val = opt[key]

    elif isinstance(opt, list):
        # Get the key-value from the cell-array
        if len(opt) % 2 != 0:
            raise ValueError("Optional input arguments should come in key-value pairs")

        keys = opt[0::2]
        vals = opt[1::2]

        if not all(isinstance(k, str) for k in keys):
            raise ValueError("Optional input arguments should come in key-value pairs, the optional input argument is invalid (should be a string)")

        hits = [i for i, k in enumerate(keys) if k.lower() == key.lower()]
        if len(hits) == 0:
            val = default
        elif len(hits) == 1:
            val = vals[hits[0]]
        else:
            raise ValueError("Multiple input arguments with the same name")

    else:
        val = default

    if val is None and default is not None and not emptymeaningful:
        val = default

    return val


In [57]:
# Depends on ADO_staircase

''' Python Code for "o_ptb_helper/volume_bayes.m" '''

def volume_bayes(cfg, sound):
    # Perform volume staircase estimation
    cfg.question_text              = ft_getopt(cfg, 'question_text', 'Haben Sie etwas gehört?')
    cfg.yes_text                   = ft_getopt(cfg, 'yes_text', 'Ja = Roter Knopf')
    cfg.no_text                    = ft_getopt(cfg, 'no_text', 'Nein = Grüner Knopf')
    cfg.db_start                   = ft_getopt(cfg, 'db_start', 40)
    cfg.db_step                    = ft_getopt(cfg, 'db_step', 1)
    cfg.db_min                     = ft_getopt(cfg, 'db_min', 20)
    cfg.db_max                     = ft_getopt(cfg, 'db_max', 120)
    cfg.max_trials                 = ft_getopt(cfg, 'max_trials', 30)
    cfg.yes_response               = ft_getopt(cfg, 'yes_response', 'yes')
    cfg.no_response                = ft_getopt(cfg, 'no_response', 'no')
    cfg.stim_delay_avg             = ft_getopt(cfg, 'stim_delay_avg', 1)
    cfg.stim_delay_jitter          = ft_getopt(cfg, 'stim_delay_jitter', 0.2)
    cfg.additional_post_stim_delay = ft_getopt(cfg, 'additional_post_stim_delay', 0.3)

    present_function = lambda bayes_cfg: volume_bayes_present(cfg, sound, bayes_cfg)
    bayes_cfg = {}
    bayes_cfg['thresh'] = cfg.db_start
    gridu = range(cfg.db_min, cfg.db_max + 1, cfg.db_step)
    thresh, sc = ADO_staircase(bayes_cfg, gridu, cfg.max_trials, present_function, 0.5, False)
    sound.db = -thresh

    return sound

In [ ]:
''' Python Code for "o_ptb_helper/volume_staircase.m" '''



In [ ]:
''' Python Code for "o_ptb_helper/private/volume_bayes_present.m" '''



In [ ]:
# DONE
''' Python Code for "calc_nstims.m" '''

# get cfg
cfg = prepare_cfg()

# get number of features
n_features = cfg['markov']['n_features']

# get ITI:
iti = cfg['markov']['iti']

# set other variables
n_conditions = 3
stims_per_sound_and_condition = 500

# calculate number of trials
n_trials = n_conditions * n_features * stims_per_sound_and_condition
# calculate duration
duration_in_min = iti * n_trials / 60

OLD:

In [ ]:
# how to play sound in psychopy:

from psychopy import sound
import numpy as np

# Set the frequency and duration of the sine wave
cur_freq = 440 # in Hz
duration = 0.1 # in seconds

# Create a time array with the specified duration
t = np.arange(0, duration, 1/sound.getSamplingRate())

# Generate the sine wave using numpy
sine_wave = np.sin(2*np.pi*cur_freq*t)

# Create a PsychoPy sound object with the generated sine wave
audio_stim = sound.Sound(sine_wave)

# Play the sound
audio_stim.play()

# Wait for the sound to finish playing
audio_stim.wait()